<a href="https://colab.research.google.com/github/Tom-Jung/Tom-Jung/blob/main/Stocks_pama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install requests beautifulsoup4 pandas


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pytz
from IPython.display import display, clear_output, HTML  # Jupyter Notebook 출력 관리

# 🔹 한국 시간대 설정
kst = pytz.timezone("Asia/Seoul")

# 🔹 네이버 금융에서 실시간 주가 및 전일 종가 가져오기
def get_naver_stock_data(stock_code):
    url = f"https://finance.naver.com/item/main.naver?code={stock_code}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # 현재가 가져오기
        price_tag = soup.select_one("p.no_today span.blind")
        current_price = int(price_tag.text.replace(",", "")) if price_tag else None

        # 전일 종가 가져오기
        prev_price_tag = soup.select("td.first span.blind")
        prev_close_price = int(prev_price_tag[0].text.replace(",", "")) if prev_price_tag else None

        return current_price, prev_close_price
    except requests.RequestException:
        return None, None

# 🔹 보유 주식 정보
portfolio = [
    {"종목명": "코스모신소재", "종목코드": "005070", "매입가": 164598, "수량": 318},
    {"종목명": "POSCO홀딩스", "종목코드": "005490", "매입가": 522538, "수량": 130},
    {"종목명": "한화오션", "종목코드": "042660", "매입가": 44000, "수량": 300},
    {"종목명": "에코프로비엠", "종목코드": "247540", "매입가": 499538, "수량": 70},
    {"종목명": "실리콘투", "종목코드": "257720", "매입가": 56300, "수량": 150}
]

# 🔹 실시간 모니터링 (Jupyter Notebook 최적화)
try:
    while True:
        clear_output(wait=True)  # 이전 출력 삭제

        # 현재 한국시간
        current_time = datetime.now(kst).strftime("[%Y-%m-%d %H:%M:%S]")

        # HTML 테이블 시작
        html_output = f"<h3>{current_time}</h3>"
        html_output += """
        <table border="1" style="border-collapse: collapse; width: 100%; text-align: center;">
            <tr style="background-color: #f2f2f2;">
                <th>종목명</th>
                <th>매입가</th>
                <th>전일 종가</th>
                <th>현재가</th>
                <th>수량</th>
                <th>현재가 × 수량</th>
                <th>손익(원)</th>
                <th>수익률(%)</th>
                <th>전일 대비(%)</th>
                <th>오늘 손익</th>  <!-- 🔹 추가된 항목 -->
                <th>포트폴리오 비중(%)</th>
            </tr>
        """

        # 🔹 총합 변수 초기화
        total_purchase = 0
        total_current = 0
        total_today_profit = 0  # 오늘 손익 총합

        stock_data = []

        for stock in portfolio:
            current_price, prev_close_price = get_naver_stock_data(stock["종목코드"])
            if current_price is None or prev_close_price is None:
                continue

            purchase_price = stock["매입가"]
            quantity = stock["수량"]

            stock_total_purchase = purchase_price * quantity
            stock_total_current = current_price * quantity
            profit_loss = stock_total_current - stock_total_purchase
            profit_percent = (profit_loss / stock_total_purchase) * 100

            # 🔹 전일 대비 상승률 및 오늘 손익 계산
            day_change_percent = ((current_price - prev_close_price) / prev_close_price) * 100
            today_profit = (current_price - prev_close_price) * quantity  # 🔹 오늘 손익 계산

            # 🔹 총합 계산
            total_purchase += stock_total_purchase
            total_current += stock_total_current
            total_today_profit += today_profit  # 🔹 오늘 손익 합산

            stock_data.append({
                "종목명": stock["종목명"],
                "매입가": purchase_price,
                "전일 종가": prev_close_price,
                "현재가": current_price,
                "수량": quantity,
                "현재가 × 수량": stock_total_current,
                "손익": profit_loss,
                "수익률": profit_percent,
                "전일 대비": day_change_percent,
                "오늘 손익": today_profit,
                "평가금액": stock_total_current
            })

        # 🔹 전체 손익 및 수익률 계산
        total_profit_loss = total_current - total_purchase
        total_profit_percent = (total_profit_loss / total_purchase) * 100 if total_purchase != 0 else 0

        # 🔹 HTML 테이블 행 추가
        for data in stock_data:
            stock_weight = (data["평가금액"] / total_current) * 100 if total_current != 0 else 0  # 포트폴리오 비중

            # 🔹 색상 적용
            profit_color = "red" if data["손익"] >= 0 else "blue"
            percent_color = "red" if data["수익률"] >= 0 else "blue"
            day_change_color = "red" if data["전일 대비"] >= 0 else "blue"
            today_profit_color = "red" if data["오늘 손익"] >= 0 else "blue"

            # 🔹 HTML 테이블 행 추가
            html_output += f"""
            <tr>
                <td>{data["종목명"]}</td>
                <td>{data["매입가"]:,}</td>
                <td>{data["전일 종가"]:,}</td>
                <td>{data["현재가"]:,}</td>
                <td>{data["수량"]:,}</td>
                <td>{data["현재가 × 수량"]:,}</td>
                <td style="color: {profit_color};">{data["손익"]:+,}</td>
                <td style="color: {percent_color};">{data["수익률"]:+.2f}%</td>
                <td style="color: {day_change_color};">{data["전일 대비"]:+.2f}%</td>
                <td style="color: {today_profit_color};">{data["오늘 손익"]:+,}</td>
                <td>{stock_weight:.2f}%</td>
            </tr>
            """

        # 🔹 총합 행 추가
        total_profit_color = "red" if total_profit_loss >= 0 else "blue"
        total_percent_color = "red" if total_profit_percent >= 0 else "blue"
        total_today_profit_color = "red" if total_today_profit >= 0 else "blue"

        html_output += f"""
        <tr style="font-weight: bold; background-color: #f9f9f9;">
            <td>총합</td>
            <td colspan="4"></td>
            <td>{total_current:,}</td>
            <td style="color: {total_profit_color};">{total_profit_loss:+,}</td>
            <td style="color: {total_percent_color};">{total_profit_percent:+.2f}%</td>
            <td>-</td>
            <td style="color: {total_today_profit_color};">{total_today_profit:+,}</td>
            <td>100.00%</td>
        </tr>
        """

        # HTML 테이블 종료
        html_output += "</table>"

        display(HTML(html_output))

        time.sleep(2)  # 2초마다 업데이트

except KeyboardInterrupt:
    print("\n📌 프로그램 종료.")
